In [1]:
import pyspark
import os
import math
import random
import sys
import pandas as pd
import numpy as np
from IPython.display import display
import re

# make sure pyspark tells workers to use python2 not 3 if both are installed\\n\",\n",
os.environ["PYSPARK_PYTHON"] = "python2"
os.environ['PYTHONPATH'] = ':'.join(sys.path)

import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)

In [2]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg.distributed import MatrixEntry
from pyspark.mllib.linalg.distributed import CoordinateMatrix
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.feature import HashingTF
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import col
from difflib import SequenceMatcher

# Obtaining the data

The full AGRIS repository is available from the [AGRIS AllegroGraph WebView](http://202.45.139.84:10035/catalogs/fao/repositories/agris) which allows us to query the repository from a sparql endpoint. However, instead of using it we have downloaded the full repository in RDF format and uploaded it to our own [Virtuoso Sparql Endpoint](http://virtuoso.udl.cat:8890/sparql). By this way, we have been able to query the repository without problems of timeout or other problems we were having using the AGRIS sparql endpoint.

Once we had our sparql endpoint working, we could start obtaining the data. It neither was an easy task because the AGRIS repository contains a huge volume of data that we could not obtain with a single query. To solve this problem we limited the queries to return one million rows per query, so we had to execute the query multiples times to obtain the full dataset.

Furthermore, another trouble to obtain the data was that some fields were lists of elements and it is a little bit difficult to obtain those fields in list format. In order to solve this problem we have divided our query into some different queries. In the first query we have obtained all the fields which contains a single value for each article.

"AGRIS QUERY GENERAL"

The other queries are one query for each field in list format we want to obtain.

"OTHER QUERIES"

In [3]:
articles_spark_DF = sqlContext.read.format("csv").load("data/Article-*.csv",header = True)
articles_lang_spark_DF = sqlContext.read.format("csv").load("data/Lang-*.csv",header = True)
authors_spark_DF = sqlContext.read.format("csv").load("data/Authors-*.csv",header = True)
subjects_spark_DF = sqlContext.read.format("csv").load("data/Sub-*.csv",header = True)
print articles_spark_DF.count()
print articles_lang_spark_DF.count()
print authors_spark_DF.count()
print subjects_spark_DF.count()

5589673
5553754
5461861
4056963


As we have said before, the AGRIS repository has a huge volume of data and we will not be able to process the whole of it. So what we will do is load the whole dataset into pandas only to obtain a sample of 10000 rows. This will be the dataset we will use to do the cleaning process.

In [4]:
df1 = pd.read_csv("data/Article-0M-1M.csv", sep=",",header=0,low_memory=False)
df2 = pd.read_csv("data/Article-1M-2M.csv", sep=",",header=0,low_memory=False)
df3 = pd.read_csv("data/Article-2M-3M.csv", sep=",",header=0,low_memory=False)
df4 = pd.read_csv("data/Article-3M-4M.csv", sep=",",header=0,low_memory=False)
df5 = pd.read_csv("data/Article-4M-5M.csv", sep=",",header=0,low_memory=False)
df6 = pd.read_csv("data/Article-5M-6M.csv", sep=",",header=0,low_memory=False)

df_ = pd.concat([df1,df2,df3,df4,df5,df6],ignore_index = True)

articles = df_.sample(n=10000)
articles.to_csv('data/current_sample.csv')
articles.sort_index(inplace=True)

df1 = pd.read_csv("data/Lang-0M-1M.csv", sep=",",header=0,low_memory=False)
df2 = pd.read_csv("data/Lang-1M-2M.csv", sep=",",header=0,low_memory=False)
df3 = pd.read_csv("data/Lang-2M-3M.csv", sep=",",header=0,low_memory=False)
df4 = pd.read_csv("data/Lang-3M-4M.csv", sep=",",header=0,low_memory=False)
df5 = pd.read_csv("data/Lang-4M-5M.csv", sep=",",header=0,low_memory=False)
df6 = pd.read_csv("data/Lang-5M-6M.csv", sep=",",header=0,low_memory=False)

df_ = pd.concat([df1,df2,df3,df4,df5,df6],ignore_index = True)

In [43]:
df1 = pd.read_csv("data/Authors-0M-1M.csv", sep=",",header=0,low_memory=False)
df2 = pd.read_csv("data/Authors-1M-2M.csv", sep=",",header=0,low_memory=False)
df3 = pd.read_csv("data/Authors-2M-3M.csv", sep=",",header=0,low_memory=False)
df4 = pd.read_csv("data/Authors-3M-4M.csv", sep=",",header=0,low_memory=False)
df5 = pd.read_csv("data/Authors-4M-5M.csv", sep=",",header=0,low_memory=False)
df6 = pd.read_csv("data/Authors-5M-6M.csv", sep=",",header=0,low_memory=False)

df_Authors_ = pd.concat([df1,df2,df3,df4,df5,df6],ignore_index = True)
df_Authors_.to_csv('data/authors.csv')

df1 = pd.read_csv("data/Sub-0M-1M.csv", sep=",",header=0,low_memory=False)
df2 = pd.read_csv("data/Sub-1M-2M.csv", sep=",",header=0,low_memory=False)
df3 = pd.read_csv("data/Sub-2M-3M.csv", sep=",",header=0,low_memory=False)
df4 = pd.read_csv("data/Sub-3M-4M.csv", sep=",",header=0,low_memory=False)
df5 = pd.read_csv("data/Sub-4M-5M.csv", sep=",",header=0,low_memory=False)

df_Subjects = pd.concat([df1,df2,df3,df4,df5],ignore_index = True)

# Cleaning process

## Duplicated articles

In [6]:
articles_spark_DF.createOrReplaceTempView("data")
# Check if there are duplicated articlesIds
print sqlContext.sql("SELECT COUNT(articleId) FROM data").show()
print sqlContext.sql("SELECT COUNT(DISTINCT(articleId)) FROM data").show()
print sqlContext.sql("SELECT articleId, Count(articleId) FROM data GROUP BY articleId HAVING COUNT(*) > 1").show()

+----------------+
|count(articleId)|
+----------------+
|         5589673|
+----------------+

None
+-------------------------+
|count(DISTINCT articleId)|
+-------------------------+
|                  5553754|
+-------------------------+

None
+------------+----------------+
|   articleId|count(articleId)|
+------------+----------------+
|CO2015000039|               4|
|DJ2012041730|               2|
|DJ2012041780|               2|
|DJ2012044257|               2|
|DJ2012044287|               2|
|DJ2012045098|               2|
|DJ2012046804|               2|
|DJ2012046887|               2|
|DJ2012046920|               2|
|DJ2012046952|               2|
|DJ2012049552|               2|
|DJ2012049843|               2|
|DJ2012054487|               2|
|DJ2012057300|               2|
|DJ2012063234|               2|
|DJ2012064631|               2|
|DJ2012067455|              12|
|DJ2012072330|               2|
|DJ2012072398|               2|
|DJ2012074987|               2|
+------------+---

In [7]:
# Join the same rows
articles_spark_DF = articles_spark_DF.dropDuplicates()
articles_spark_DF.createOrReplaceTempView("data")
print sqlContext.sql("SELECT COUNT(articleId) FROM data").show()
print sqlContext.sql("SELECT COUNT(DISTINCT(articleId)) FROM data").show()
print sqlContext.sql("SELECT articleId, Count(articleId) FROM data GROUP BY articleId HAVING COUNT(*) > 1").show()

+----------------+
|count(articleId)|
+----------------+
|         5574251|
+----------------+

None
+-------------------------+
|count(DISTINCT articleId)|
+-------------------------+
|                  5553754|
+-------------------------+

None
+--------------+----------------+
|     articleId|count(articleId)|
+--------------+----------------+
|  FR2001000690|               8|
|  RU2015H08472|               2|
|  KR2015000782|               4|
|  DJ2012084336|               2|
|  FR2001000569|               8|
|  FI2007188479|               4|
|  DJ2012075801|               2|
|  DJ2012076174|               2|
|     US8206741|               8|
|  DJ2012064631|               2|
|  BG2015100016|               4|
|     US8210952|               4|
|US201300869572|               4|
|  DJ2012046804|               2|
|  FI2007029186|               4|
|  DJ2012054487|               2|
|  DJ2012078767|               2|
|  DJ2012049552|               2|
|  DJ2012044257|               2|
|  DJ

In [8]:
def equalArticles(x,y):
    if not isinstance(x.issued,list):
        x = Row(articleId=x.articleId,issued=[x.issued],submitted=[x.submitted],conferenceTitle=[x.conferenceTitle],journalTitle=[x.journalTitle],journalIssn=[x.journalIssn])
    if not isinstance(y.issued,list):
        y = Row(articleId=y.articleId,issued=[y.issued],submitted=[y.submitted],conferenceTitle=[y.conferenceTitle],journalTitle=[y.journalTitle],journalIssn=[y.journalIssn])
    return Row(articleId=x.articleId, issued=list(set(x.issued) | set(y.issued)), submitted=list(set(x.submitted) | set(y.submitted)), conferenceTitle=list(set(x.conferenceTitle) | set(y.conferenceTitle)), journalTitle=list(set(x.journalTitle) | set(y.journalTitle)), journalIssn=list(set(x.journalIssn) | set(y.journalIssn)))

def mapPhase(a):
    y = a[1]
    #if isinstance(y.issued,list):
        #return Row(articleId=y.articleId,issued='|'.join([unicode(x, "utf-8") for x in y.issued]),submitted='|'.join([unicode(str(x), "utf-8") for x in y.submitted]),conferenceTitle='|'.join([unicode(str(x), "utf-8") for x in y.conferenceTitle]),journalTitle='|'.join([unicode(str(x), "utf-8") for x in y.journalTitle]),journalIssn='|'.join([unicode(str(x), "utf-8") for x in y.journalIssn]))

    return y

schema = StructType([StructField("articleId", StringType(), True),StructField("issued", StringType(), True),StructField("submitted", StringType(), True),StructField("conferenceTitle", StringType(), True),StructField("journalTitle", StringType(), True),StructField("journalIssn", StringType(), True)])

In [9]:
articles_spark_RDD = articles_spark_DF.rdd.map(lambda x: (x.articleId,x)).reduceByKey(equalArticles).map(mapPhase)
articles_spark_DF = sqlContext.createDataFrame(articles_spark_RDD,schema)
articles_spark_DF.createOrReplaceTempView("data")

In [10]:
print sqlContext.sql("SELECT COUNT(articleId) FROM data").show()
print sqlContext.sql("SELECT COUNT(DISTINCT(articleId)) FROM data").show()
print sqlContext.sql("SELECT articleId, Count(articleId) FROM data GROUP BY articleId HAVING COUNT(*) > 1").show()

+----------------+
|count(articleId)|
+----------------+
|         5553754|
+----------------+

None
+-------------------------+
|count(DISTINCT articleId)|
+-------------------------+
|                  5553754|
+-------------------------+

None
+---------+----------------+
|articleId|count(articleId)|
+---------+----------------+
+---------+----------------+

None


## Join articles data and language data

In [11]:
articles_spark_DF = \
articles_spark_DF.join(articles_lang_spark_DF, articles_spark_DF.articleId == articles_lang_spark_DF.articleId)\
.drop(articles_lang_spark_DF.articleId)

articles_spark_DF.createOrReplaceTempView("data")
print sqlContext.sql("SELECT * FROM data WHERE articleId = 'FR2001000690'").show()
#.rdd.filter(lambda x: isinstance(x.issued,list)).toDF()\

+------------+---------------+------------+---------------+--------------------+--------------------+--------+
|   articleId|         issued|   submitted|conferenceTitle|        journalTitle|         journalIssn|language|
+------------+---------------+------------+---------------+--------------------+--------------------+--------+
|FR2001000690|[mar2001, 1999]|[2002, 2001]|         [null]|[Game and Wildlif...|[1622-7662, 0750-...| eng|fre|
+------------+---------------+------------+---------------+--------------------+--------------------+--------+

None


## Get sampled data for the cleaning process

In [40]:
articles = articles_spark_DF.where("issued NOT LIKE '[%' AND language NOT LIKE '%|%'").sample(False,0.002,42).toPandas()

In [13]:
articles.rename(columns={'conferenceTitle':'conferenceData', 'journalTitle':'journalData'}, inplace=True)

## Extract conference information:

We want to classify all the conferences, setting the same ID in the same conferences.

Also from conferences, we are interested in the year, in order to compare it with the other years in the data.

In [14]:
# Tenen conferencia
len(articles[articles.conferenceData.notnull()])

261

In [15]:
conferenceDF =  pd.DataFrame(articles.conferenceData)

### Get the year from the data
First, we validate if all the dates have a year

In [16]:
# Que els quatre digits seguits siguin any
# Primer miro si hi han mes de un grup de 4 digits seguits ens les dades
pattern = r'^.*\d{4}.*\d{4}.*$'
np.sum(conferenceDF.conferenceData[conferenceDF.conferenceData.notnull()].str.contains(pattern,na=False))

11

In [17]:
# Si n'hi ha, mirem que siguin el mateix any. Mostrem els que son diferents valors. Aquetst els despreciarem
pattern = r'^.*(\d{4}).*(\d{4}).*$'
moreThanOne = conferenceDF.conferenceData[conferenceDF.conferenceData.notnull()].str.extract(pattern,expand=False)
moreThanOneDifferents = moreThanOne[moreThanOne[0].notnull() & (moreThanOne[0] != moreThanOne[1])]

In [18]:
moreThanOneDifferents.index

Int64Index([2277, 7272], dtype='int64')

In [19]:
# Si el resultat és 0 també s'ha de comprovar que totes tinguin 4 digits seguits
pattern = r'^.*\d{4}.*$'
len(conferenceDF.conferenceData[conferenceDF.conferenceData.notnull()]) - np.sum(conferenceDF.conferenceData[conferenceDF.conferenceData.notnull()].str.contains(pattern,na=False))

5

In [20]:
# Busquem les que no tenen els anys amb format correcte
badYearFormat = conferenceDF.conferenceData[conferenceDF.conferenceData.notnull()].str.contains(pattern,na=False)
badYearFormat = badYearFormat[badYearFormat == False]
conferenceDF.conferenceData[conferenceDF.conferenceData.index.isin(badYearFormat.index)]

526                           "Seminario ""Medio Ambiente
1448    "Tagung der DVG. Fachgruppe ""Rinderkrankheite...
4159     Aggiornamenti di nutrizione avicola. [np]. [nd].
6426    "Tagung der DVG. Fachgruppe ""Kleintierkrankhe...
9336    Seminario Avances en propagación vegetativa pa...
Name: conferenceData, dtype: object

In [21]:
# Obtenim l'any i eliminem els que hi han a moreThanOneDifferents
pattern = r'^.*(\d{4}).*$'
conferenceDF['conferenceDateYear'] = conferenceDF.conferenceData.str.extract(pattern, expand=False)
conferenceDF.conferenceDateYear[conferenceDF.conferenceDateYear.index.isin(moreThanOneDifferents.index)] = np.nan
conferenceDF.conferenceDateYear = pd.to_numeric(conferenceDF.conferenceDateYear, errors='coerce')
len(conferenceDF.conferenceDateYear[conferenceDF.conferenceDateYear.notnull()])

254

In [22]:
articles["conferenceYear"] = conferenceDF.conferenceDateYear

## Extract year and month from issued date

The field 'issued' shows when the article was published and it comes with some different formats incliding the year and sometimes the month. 

In [23]:
issuedYear = articles.issued.str.extract("(\d{4}|[0,3-9]\d)", expand=False)
issuedYear = pd.to_numeric(issuedYear, errors='coerce')

In [24]:
def completeYears(year):
    century = 1900 if year > 40 else 2000
    return century + year
issuedYear[issuedYear < 99] = issuedYear[issuedYear < 99].apply(completeYears)

In [25]:
articles['issuedYear'] = issuedYear

In some cases, the month comes in a numeric way and other times it comes within its abbreviation. We will unify it converting the abbreviations to numerical months.

In [26]:
issuedMonthText = articles.issued.str.extract("(ene|jan|feb|mar|apr|abr|mai|may|jun|jul|aug|ago|sept|sep|oct|nov|dic|dec)", expand=False)
issuedMonthNumeric = articles.issued.str.extract("\D(\d|1[1-2])\D?$", expand=False)
issuedMonth = issuedMonthText.fillna(issuedMonthNumeric)

In [27]:
issuedMonth = issuedMonth.replace('ene|jan', '1', regex=True)
issuedMonth = issuedMonth.replace('feb', '2')
issuedMonth = issuedMonth.replace('mar', '3')
issuedMonth = issuedMonth.replace('apr|abr', '4', regex=True)
issuedMonth = issuedMonth.replace('mai|may', '5', regex=True)
issuedMonth = issuedMonth.replace('jun', '6')
issuedMonth = issuedMonth.replace('jul', '7')
issuedMonth = issuedMonth.replace('ago|aug', '8', regex=True)
issuedMonth = issuedMonth.replace('sept|sep', '9', regex=True)
issuedMonth = issuedMonth.replace('oct', '10')
issuedMonth = issuedMonth.replace('nov', '11')
issuedMonth = issuedMonth.replace('dic|dec', '12', regex=True)
issuedMonth = issuedMonth.fillna('6')

In [28]:
articles['issuedMonth'] = issuedMonth

Finally, we will check that we have specified an issuedYear and issuedMonth for each row of the sample dataset.

In [29]:
articles.issuedYear[articles.issuedYear.isnull() == True].count()

0

In [30]:
articles.issuedMonth[articles.issuedMonth.isnull() == True].count()

0

## Check correctness of issued, submitted year and conference year

Once we have obtained the issued year, we can check that the article had been issued before it had been submitted to the AGRIS repository ('submitted' field).

In [32]:
articles.ix[articles.submitted - articles.issuedYear < 0, 'issuedYear'] = articles[articles.submitted - articles.issuedYear < 0].submitted

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U21') dtype('<U21') dtype('<U21')

In [ ]:
articles[articles.submitted - articles.issuedYear < 0].count()

In [ ]:
# Check correctness of conference year
articles.conferenceYear[(articles.conferenceYear < 1970) | (articles.conferenceYear > 2020)]

In [ ]:
articles.articleId[articles.conferenceYear.notnull() & (articles.conferenceYear > articles.submitted)].count()

## Removing misstyped journal names

Our dataset contains the journal where some articles have been published. Specifically, there is a field specifying the name of the journal and another field specifying the ISSN of the journal. The problem is that sometimes the journal name does not match for a particular ISSN, which is obviously an error. To solve this problem we associate the most frequent journal name for each ISSN to all the articles of this ISSN.

In [33]:
articles.journalData = articles.journalIssn.map(articles.groupby('journalIssn').journalData.agg(lambda x:x.value_counts().index[0]).to_dict())

In [34]:
articles.journalIssn = articles.journalData.map(articles.groupby('journalData').journalIssn.agg(lambda x:x.value_counts().index[0]).to_dict())

In [35]:
articles.groupby('journalIssn').journalData.value_counts()

journalIssn                                                              journalData                                                                                                      
 otdelenie biologicheskikh nauk"                                         "Akhboroti Akademiiai fankhoi RSS Tochikiston, shu""bai fankhoi biologi = Izvestiia Akademii nauk Tadzhikskoi SSR     1
0001-1541                                                                AIChE journal                                                                                                         2
0001-2351                                                                Transactions of the ASAE (USA)                                                                                       10
0001-4079                                                                Bulletin de l'Academie Nationale de Medecine (France)                                                                 1
0001-4087                                

In [36]:
articles.groupby('journalData').journalIssn.value_counts()

journalData                                                                                                        journalIssn                                                            
 F.R.). 14-15 Feb 1989."                                                                                           Deutsche Veterinaermedizinische Gesellschaft e.V. DVG , Tagungsberichte    2
"Akhboroti Akademiiai fankhoi RSS Tochikiston, shu""bai fankhoi biologi = Izvestiia Akademii nauk Tadzhikskoi SSR   otdelenie biologicheskikh nauk"                                           1
"Moscosoa : Contribuciones cientificas del Jardin Botanico Nacional ""Dr. Rafael M. Moscoso"""                     0254-6442                                                                  1
(Allattenyesztes es Takarmanyozas)(Hungary)                                                                        0230-1814                                                                  1
A la Pointe de l'Elevage (France)            

## Languages correctness

We have detected that sometimes language codes are capitalized, so we will transform all codes to lower case. Additionally, we want to be sure that the codes are correct so we have found a dataset with all the language codes and the language which is representing the code in English. Therefor, we will verify the language codes with the language codes dataset and we will fill another column with the language in English for a more visual understanding.

In [37]:
articles.language = articles.language.str.lower()

In [38]:
articles.groupby('language').language.value_counts()

language         language       
afr              afr                   3
ara              ara                   5
bel              bel                   2
bem              bem                   6
bul              bul                  43
cat              cat                   1
chi              chi                 106
cze              cze                  55
dan              dan                  53
dut              dut                  84
eng              eng                7254
eng|ara          eng|ara               1
eng|dan          eng|dan               2
eng|esp          eng|esp               2
eng|fre          eng|fre               7
eng|fre|esp      eng|fre|esp           9
eng|fre|spa      eng|fre|spa           1
eng|ger          eng|ger               4
eng|ger|fre      eng|ger|fre           1
eng|ger|fre|ita  eng|ger|fre|ita       1
eng|hun          eng|hun               1
eng|may          eng|may               1
eng|nor          eng|nor               1
eng|por          eng|por

In [ ]:
langs = pd.read_csv("data/language-codes-3b2.csv", sep=",",header=0,low_memory=False)

In [ ]:
def extractLanguageName(lang):
    langName = langs.ix[langs['alpha3-b'] == lang, 'English']
    if(langName.count() > 0):
        return langName.iloc[0]
    else:
        return np.nan

In [ ]:
articles['languageName'] = articles.language.map(extractLanguageName)

In [ ]:
langs.ix[langs['alpha3-b'] == 'fre', 'English'].iloc[0]

In [ ]:
articles[articles.languageName.isnull() == True].language.value_counts()

In [ ]:
articles.ix[articles.language == 'esp','languageName'] = 'Spanish; Castilian'
articles.ix[articles.language == 'esp','language'] = 'spa'
articles.ix[articles.language == 'fra','languageName'] = 'French'
articles.ix[articles.language == 'fra','language'] = 'fre'

### Set conference Ids

In [ ]:
conferenceDF["conferenceId"] = np.nan

In [ ]:
# https://docs.python.org/2.4/lib/sequence-matcher.html

# Return a measure of the sequences' similarity as a float in the range [0, 1].
# Where T is the total number of elements in both sequences, and M is the number of matches, this is 2.0*M / T. 
# Note that this is 1.0 if the sequences are identical, and 0.0 if they have nothing in common.
# This is expensive to compute if get_matching_blocks() or get_opcodes() hasn't already been called, 
# in which case you may want to try quick_ratio() or real_quick_ratio() first to get an upper bound.

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
# https://en.wikipedia.org/wiki/Jaccard_index

# The Jaccard Similarity between two sets A, B is
# |Intersection(A, B)| / |Union(A, B)|

def jaccardSimilarity (totalCommon, totalA, totalB):
    union = totalA + totalB - totalCommon
    return totalCommon * 1.0 / union

def intersect(a,b):
    res = []
    for x in a:
        if x in b:
            res.append(x)
    return res

def parseData(a,b):
    val = jaccardSimilarity(len(intersect(a[0],b[0])),len(a[0]),len(b[0]))
    return (a[1],b[1],val)

In [ ]:
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
# https://pypi.python.org/pypi/editdistance

# Levenshtein distance algorithm. This tells us the number of edits needed to turn one string into another. 

def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

In [ ]:
# DIMSUM

# http://spark.apache.org/docs/2.0.2/api/python/pyspark.mllib.html#module-pyspark.mllib.linalg.distributed
# https://es.wikipedia.org/wiki/Similitud_coseno
# https://databricks.com/blog/2014/10/20/efficient-similarity-algorithm-now-in-spark-twitter.html

def lexicoOrder(s):
    s.sort(key=lambda item: (len(item), item))
    return s

htf = HashingTF()

def parseHash(e):
    return htf.indexOf(e)

def parseStringToHashItemsList(s):
    res = []
    for e in lexicoOrder(s.split(" ")):
        res.append(parseHash(e))
    return res
    
rows = sc.parallelize(strings).map(parseStringToHashItemsList)

    mat = RowMatrix(rows)

def parseVec(v,idx):
    vec = []
    for idy, el in enumerate(v.toArray()):
        vec.append(MatrixEntry(idy,idx,el))
    return vec
  

def transpose(m):
    rows = m.rows
    rdd = rows.zipWithIndex().flatMap(lambda (vec, idx): [x for x in parseVec(vec,idx)])
    return rdd

coordMat = CoordinateMatrix(transpose(mat))
rowMatrix = coordMat.toRowMatrix()
mat = rowMatrix

sims = mat.columnSimilarities() # sims es del tipus CoordinateMatrix
# sims.entries és un RDD de MatrixEntry del format (long,long,float)

transformedRDD = sims.entries.map(lambda e: np.array([e.i,e.j,e.value]))
print transformedRDD.collect()

In [ ]:
# https://en.wikipedia.org/wiki/MinHash
# Implemented in the last scala or java API 
# http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.feature.MinHashLSH
# http://spark.apache.org/docs/2.1.0/api/java/org/apache/spark/ml/feature/MinHashLSH.html



In [ ]:
def similarConference(a,b):
    if a["conferenceDateYear"] != b["conferenceDateYear"]:
        return False
    return similar(a["conferenceData"],b["conferenceData"]) > 0.6

def conferenceComput(cdf):
    ID = 1
    diff = 0
    addNew = True
    for idx, x in cdf[cdf.conferenceData.notnull()].iterrows():
        if ID == 1:
            cdf.loc[idx,"conferenceId"] = ID
            ID += 1
            continue
        for idy, y in cdf[cdf.conferenceId.notnull()].iterrows():
            if similarConference(x,y):
                #cdf.loc[idx,"conferenceId"] = cdf.loc[idy,"conferenceId"]
                cdf.set_value(idx,"conferenceId",cdf.loc[idy,"conferenceId"])
                addNew = False
                diff += 1
                break
        if addNew:
            cdf.loc[idx,"conferenceId"] = ID
            ID += 1
        addNew = True
    return diff
conferenceComput(conferenceDF)

In [ ]:
articles["ConferenceId"] = conferenceDF.conferenceId

# Coauthors networks

In [44]:
authorsLines = sc.textFile("data/authors.csv")
header = authorsLines.first()
rows = authorsLines.filter(lambda line: line != header)
authors = rows.flatMap(lambda line: re.findall('"([^"]*)"', line)).map(lambda a: a.split('|')) 

In [45]:
authorsLines.count()

5576414

## Coauthorship networks

As the final objective of this project is to disambiguate author names which corresponds to different persons, we will do it by obtaining the coauthorship networks of a given author and toughen the different networks with other parameters. 

The first step of this process is to extract the coauthorship networks of a given author. To do it, we filter the rows searching for the given author and we obtain the network representing the relations of this author.

In [54]:
import itertools   
keyToSearch = 'Zidorn, C.'#'Yamaoka, Y.'#'Zidorn, C.'#'Ellmerer, E.P.'#'Congress.'
filteredAuthors = authors.map(set).filter(lambda a: keyToSearch in a).map(lambda a: a.difference({keyToSearch}))
filteredAuthors.count()

20

In [55]:
coauthors = filteredAuthors.flatMap(lambda a: [[tuple(sorted(coauth)), 1] for coauth in itertools.combinations(a, 2)])
coauthorsMap = coauthors.reduceByKey(lambda x, y: x + y)
coauthorsList = coauthorsMap.map(lambda (key,val): (key[0],key[1],val))

In [56]:
def yieldFunc(list):
    for item in list:
        yield item
coauthorsunique = filteredAuthors.flatMap(yieldFunc).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [57]:
def toCSVLine(data):
    return ';'.join(str(d) for d in data)

lines = coauthorsunique.map(toCSVLine)
lines.saveAsTextFile("data/coauthorsunique/" + keyToSearch.split(",")[0]);

lines = coauthorsList.map(toCSVLine)
lines.saveAsTextFile("data/coauthorslist/" + keyToSearch.split(",")[0]);

## Most frequent coauthors

We have used the algorithm FP-Growth to obtain the most frequent coauthor networks oof two or more authors. Later on, we will use this results to toughen the relation between these author on the particular author's coauthorship networks graph. By this way, when we will see a strong line between two authors on a coauthorship network we will rely on this relation and it will be another factor to determine that the author is only one person for this coauthorship network.

In [58]:
coauthors = authors.map(lambda a: list(set(a)))

In [59]:
from pyspark.mllib.fpm import FPGrowth

model = FPGrowth.train(coauthors, minSupport=0.00001, numPartitions=4)
rddfreqitemsets = model.freqItemsets()

In [60]:
result = rddfreqitemsets.filter(lambda x  : len(x.items) >= 2).collect()

Unfortunately, most of the authors obtained from this algorithm are Universities or other strange names that does not correspond to an author name.

In [61]:
result

[FreqItemset(items=[u'O., College of Education', u'O.T., Afe Babalola University'], freq=54),
 FreqItemset(items=[u'O., College of Education', u'O.T., Afe Babalola University', u'A.A., Afe Babalola University'], freq=54),
 FreqItemset(items=[u'O., College of Education', u'A.A., Afe Babalola University'], freq=54),
 FreqItemset(items=[u'Iinuma, M.', u'Tanaka, T.'], freq=91),
 FreqItemset(items=[u'T.A., Lagos State Polytechnic', u'A.S., Lagos State Polytechnic'], freq=54),
 FreqItemset(items=[u'O.A., Lagos State Polytechnic', u'Y.O., Lagos State Polytechnic'], freq=54),
 FreqItemset(items=[u'O.A., Lagos State Polytechnic', u'Y.O., Lagos State Polytechnic', u'D.O., Lagos State Polytechnic'], freq=54),
 FreqItemset(items=[u'O.A., Lagos State Polytechnic', u'D.O., Lagos State Polytechnic'], freq=54),
 FreqItemset(items=[u'Degraer, S.', u'Vincx, M.'], freq=96),
 FreqItemset(items=[u'Nguyen, Vietnam Academy of Science and Technology', u'Tran Dang, Hiroshima University'], freq=54),
 FreqItemse